# Cython!

### What is this?

It seems that everywhere you see criticism of Python, it always has at least something about its execution speed.

While there are a variety of options for optimising Python code, the Cython project is (IMHO) the best tradeoff between results and headaches.

For starters:
```
Cython != CPython
```
Cython is a superset language of Python which compiles to C (or C++, apparently).

This gives us some cool features:
- All Python is valid Cython
- Static compilation gives us access to compiler optimisation
- Cython provides optional *static typing*, which gives the compiler something to really optimise over. This is what we're really here for :D
- Because we end up in C land, calling C code is much more straightforward (apparently, I haven't done this before)

~~Unfortunately, Cython's documentation is hit and miss, and its community seems not to be particularly active.~~

Cython's documentation is improving, but it's still difficult to get into, and difficult to learn how to ~~use~~ *abuse* the way it works.

So we're going to look at some basics of working with Cython and the (actually pretty great) tools it provides.

The main point we want to explore here is how Cython is able to work as a strict superset of Python.

This is actually pretty simple: when Cython can't generate C code for some part of a statement it will pass off to the Python interpreter.

But Cython *only* provides performance improvements when it isn't moving between compiled code and the Python interpreter (what a surprise!).

Avoiding this back and forth between the interpreter and the compiled code is not always straightforward~~, and rarely explained well in the official documentation~~.

### NOTE!

We're not going to look into using Cython in an actual project here.

There are a few ways of integrating Cython into a project, and a whole host of options to configure both the Python -> C compilation and the final C code compilation.

# Today's example :P

I've pulled a tidy little example from my PhD work that really benefitted from a Cython optimisation.

When finished this went from a multiple hour bottleneck to taking only a couple of minutes, making it the fastest part of the program.

We're going to be computing the second-order Møller-Plesset perturbation energy:

$$E_{MP2} = \sum^\mathrm{occ}_{ab} \sum^\mathrm{virt}_{rs} \frac{(ab||rs)}{\epsilon_a + \epsilon_b - \epsilon_r - \epsilon_s}$$

But not even, because we'll just randomly generate some input :P

This is a massive pain to compute because its a complex 4 index loop.

Python sucks at looping like this.

### some setup

In [1]:
%load_ext cython
import numpy as np

In [2]:
total_fns = 50
num_electrons = 6
integrals = np.random.random_sample(4 * [total_fns])
energies = np.random.random_sample([total_fns])

## CPython

Here's a standard implementation in Python for computing this value.

The optimal algorithm for this uses a complex iteration scheme that exploits symmetry in the input array `integrals`.

This does not translate well to Python because it can't be expressed in terms of a comprehension, forcing us to use a large Python `for` loop.

In [3]:
def mp2_correction(electrons, integrals, orb_energies):
    """Compute the second order Moller Plesset correction.

    Args:
        electrons (int): Number of electrons in the system.
        integrals (np.array): 4-index array of double bar integrals in
            molecular orbital space.
        orb_energies (np.array): Array of orbital energies.
    """

    total_fns = len(integrals)
    correction = 0
    # pylint: disable=invalid-name
    for a in range(electrons):
        for b in range(a, electrons):
            for r in range(electrons, total_fns):
                for s in range(r, total_fns):
                    correction += (
                        integrals[a, r, b, s]**2 / (
                            orb_energies[a]
                            + orb_energies[b]
                            - orb_energies[r]
                            - orb_energies[s]
                        )
                    )

    return correction

In [4]:
%%timeit -n 100
mp2_correction(num_electrons, integrals, energies)

100 loops, best of 3: 37.2 ms per loop


## Naïve Cython

Just straight running Cython over some Python code can get a speed boost of around 20%.

The --annotate option is a crucial tool that Cython provides.

It shows us the C code that Cython generates, but more importantly shows us how much it has called back to the Python interpreter.

Unfortunately we don't really get any boost here because Cython isn't able to pull apart the Python loop construct, which is the crux of this problem.

In [5]:
%%cython --annotate
def mp2_correction(electrons, integrals, orb_energies):
    """Compute the second order Moller Plesset correction.

    Args:
        electrons (int): Number of electrons in the system.
        integrals (np.array): 4-index array of double bar integrals in
            molecular orbital space.
        orb_energies (np.array): Array of orbital energies.
    """

    total_fns = len(integrals)
    correction = 0
    # pylint: disable=invalid-name
    for a in range(electrons):
        for b in range(a, electrons):
            for r in range(electrons, total_fns):
                for s in range(r, total_fns):
                    correction += (
                        integrals[a, r, b, s]**2 / (
                            orb_energies[a]
                            + orb_energies[b]
                            - orb_energies[r]
                            - orb_energies[s]
                        )
                    )

    return correction

In [6]:
%%timeit -n 100
mp2_correction(num_electrons, integrals, energies)

100 loops, best of 3: 27.5 ms per loop


# Simple typing

Static typing! That's where the magic happens, so adding some will help, right?

### NOTE!
A number of popular libraries provide Cython hooks. To use them we need to `cimport` them.

If you're writing Cython, then you are already using these libraries. Make sure to use the Cython hooks!

In [7]:
%%cython --annotate
cimport numpy as np

def mp2_correction(electrons, integrals, orb_energies):
    """Compute the second order Moller Plesset correction.

    Args:
        electrons (int): Number of electrons in the system.
        integrals (np.array): 4-index array of double bar integrals in
            molecular orbital space.
        orb_energies (np.array): Array of orbital energies.
    """
    cdef int a, b, r, s

    total_fns = len(integrals)
    correction = 0
    # pylint: disable=invalid-name
    for a in range(electrons):
        for b in range(a, electrons):
            for r in range(electrons, total_fns):
                for s in range(r, total_fns):
                    correction += (
                        integrals[a, r, b, s]**2 / (
                            orb_energies[a]
                            + orb_energies[b]
                            - orb_energies[r]
                            - orb_energies[s]
                        )
                    )

    return correction

In [8]:
%%timeit -n 100
mp2_correction(num_electrons, integrals, energies)

100 loops, best of 3: 25 ms per loop


# Don't write your Cython code at 11pm kids!

In [9]:
%%cython --annotate
cimport numpy as np

def mp2_correction(electrons, integrals, orb_energies):
    """Compute the second order Moller Plesset correction.

    Args:
        electrons (int): Number of electrons in the system.
        integrals (np.array): 4-index array of double bar integrals in
            molecular orbital space.
        orb_energies (np.array): Array of orbital energies.
    """
    cdef int a, b, r, s
    # this bit's important :)
    cdef int num_electrons, total_fns
    cdef np.double_t correction
    
    num_electrons = electrons

    total_fns = len(integrals)
    correction = 0
    # pylint: disable=invalid-name
    for a in range(num_electrons):
        for b in range(a, num_electrons):
            for r in range(num_electrons, total_fns):
                for s in range(r, total_fns):
                    correction += (
                        integrals[a, r, b, s]**2 / (
                            orb_energies[a]
                            + orb_energies[b]
                            - orb_energies[r]
                            - orb_energies[s]
                        )
                    )

    return correction

In [10]:
%%timeit -n 100
mp2_correction(num_electrons, integrals, energies)

100 loops, best of 3: 25.2 ms per loop


# ~~Array typing~~ Memory views

Python iterables are difficult objects, because they provide a lot of nice features and safety to the programmer.

C doesn't give you safety though...

In order to ensure that you're Cython code behaves like the Python you expect, Cython does a lot of extra work to give you those features. This slows us down a lot.

*Memory views* let us tell Cython that there's a chunk of memory underneath that it can access, and how the Python objects are mapped into it.

In [11]:
%%cython --annotate
cimport numpy as np

def mp2_correction(electrons, integrals, orb_energies):
    """Compute the second order Moller Plesset correction.

    Args:
        electrons (int): Number of electrons in the system.
        integrals (np.array): 4-index array of double bar integrals in
            molecular orbital space.
        orb_energies (np.array): Array of orbital energies.
    """
    cdef int a, b, r, s
    cdef int num_electrons, total_fns
    cdef np.double_t correction
    cdef np.double_t[:,:,:,:] cython_integrals
    cdef np.double_t[:] cython_energies

    num_electrons = electrons
    cython_integrals = integrals
    cython_energies = orb_energies

    total_fns = len(integrals)
    correction = 0
    # pylint: disable=invalid-name
    for a in range(num_electrons):
        for b in range(a, num_electrons):
            for r in range(num_electrons, total_fns):
                for s in range(r, total_fns):
                    correction += (
                        cython_integrals[a, r, b, s]**2 / (
                            cython_energies[a]
                            + cython_energies[b]
                            - cython_energies[r]
                            - cython_energies[s]
                        )
                    )

    return correction

In [12]:
%%timeit -n 100
mp2_correction(num_electrons, integrals, energies)

100 loops, best of 3: 210 µs per loop


# Cython directives

We've still got a little bit of Python interaction within our loop.

Cython injects a lot of extra code into the C that it generates to make it behave like Python.

Also known as: Cython stops you from shooting a hole in your foot.

But if we really want to fly then we need to take the training wheels off.

Cython provides a number of directives for its Python -> C compilation to do this to varying degrees.

In [13]:
%%cython --annotate
import cython
cimport numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def mp2_correction(electrons, integrals, orb_energies):
    """Compute the second order Moller Plesset correction.

    Args:
        electrons (int): Number of electrons in the system.
        integrals (np.array): 4-index array of double bar integrals in
            molecular orbital space.
        orb_energies (np.array): Array of orbital energies.
    """
    cdef int a, b, r, s
    cdef np.double_t correction
    cdef np.double_t[:,:,:,:] cython_integrals
    cdef np.double_t[:] cython_energies
    cdef int cython_electrons, total_fns
    
    cython_integrals = integrals
    cython_energies = orb_energies
    cython_electrons = electrons

    total_fns = len(integrals)
    correction = 0
    # pylint: disable=invalid-name
    for a in range(cython_electrons):
        for b in range(a, cython_electrons):
            for r in range(cython_electrons, total_fns):
                for s in range(r, total_fns):
                    correction += (
                        cython_integrals[a, r, b, s]**2 / (
                            cython_energies[a]
                            + cython_energies[b]
                            - cython_energies[r]
                            - cython_energies[s]
                        )
                    )

    return correction

Unfortunately we don't see to much here, but this can make massive dents in certain situations.

In [14]:
%%timeit -n 100
mp2_correction(num_electrons, integrals, energies)

100 loops, best of 3: 200 µs per loop


# The end!

That's all we have for a quick intro to some of the basic tricks you need to know to get the most out of Cython.

:D